In [ ]:
import math
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup 

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
cols = ['title', 'company', 'location', 'salary', 'days_since_posted', 'link', 'night_shift', 'sql', 'excel', 'tableau', 
        'power_bi', 'python', 'r', 'career_level', 'qualification', 'years_of_experience', 'job_type', 'specialization', 
        'subspecialty', 'registration_no', 'company size', 'ave_process_time', 'industry', 'benefits_and_others']
extracted_data = []
job_info = []
    
# Additional information section
def job_addtl_info(job_post_soup_prettify):
    job_post_div = job_post_soup_prettify.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw6r pmwfa50 pmwfa57'})

    for job_addtl_desc in job_post_div:
        skills = job_addtl_desc.text

        if 'career level' in skills.lower():
            pattern = re.compile('Career Level', re.IGNORECASE)
            job_info[13] = pattern.sub('', skills).strip()             
        elif 'qualification' in skills.lower():
            pattern = re.compile('Qualification', re.IGNORECASE)
            job_info[14] = pattern.sub('', skills).strip()             
        elif 'years of experience' in skills.lower():
            pattern = re.compile('Years of Experience', re.IGNORECASE)
            job_info[15] = pattern.sub('', skills).strip()             
        elif 'job type' in skills.lower():
            pattern = re.compile('Job Type', re.IGNORECASE)
            job_info[16] = pattern.sub('', skills).strip()             
        elif 'job specializations' in skills.lower():
            pattern = re.compile('Job Specializations', re.IGNORECASE)
            specializations = pattern.sub('', skills).split(',')
            job_info[17] = specializations[0].strip()
            job_info[18] = specializations[1].strip()    
        elif 'registration no.' in skills.lower():
            pattern = re.compile('registration no.', re.IGNORECASE)
            job_info[19] = pattern.sub('', skills).strip()
        elif 'company size' in skills.lower():
            pattern = re.compile('company size', re.IGNORECASE)
            job_info[20] = pattern.sub('', skills).strip()
        elif 'average processing time' in skills.lower():
            pattern = re.compile('average processing time', re.IGNORECASE)
            job_info[21] = pattern.sub('', skills).strip()
        elif 'industry' in skills.lower():
            pattern = re.compile('industry', re.IGNORECASE)
            job_info[22] = pattern.sub('', skills).strip()
        elif 'benefits & others' in skills.lower():
            pattern = re.compile('benefits & others', re.IGNORECASE)
            job_info[23] = pattern.sub('', skills).strip()
            whitespace = re.compile(r'\s+')
            job_info[23] = whitespace.sub(' ', job_info[23]).strip()            

# Job description section containing skills
def job_skills_desc(job_post_soup_prettify):
    job_post_div = job_post_soup_prettify.findAll('div', attrs={'data-automation': 'jobDescription'})

    for job_desc in job_post_div:
        description = job_desc.text
        
        if 'night shift' in description.lower():
            job_info[6] = 'Yes'
        else:
            job_info[6] = 'No'

        if 'sql' in description.lower():
            job_info[7] = 'Yes'
        else:
            job_info[7] = 'No'

        if 'excel' in description.lower():
            job_info[8] = 'Yes'
        else:
            job_info[8] = 'No'

        if 'tableau' in description.lower():
            job_info[9] = 'Yes'
        else:
            job_info[9] = 'No'

        if 'power bi' in description.lower():
            job_info[10] = 'Yes'
        else:
            job_info[10] = 'No'

        if 'python' in description.lower():
            job_info[11] = 'Yes'
        else:
            job_info[11] = 'No'

        if re.search("[^a-zA-Z]R[^a-zA-Z]", description):
            job_info[12] = 'Yes'
        else:
            job_info[12] = 'No'
                        
# Page containing the job list
def jobs_extract(job_list):  
    for job_post in job_list:        
        global job_info 
        job_info = ['none'] * len(cols)
        
        # Job Title
        job_info[0] = job_post.find('div', attrs={'class': 'z1s6m00 l3gun70 l3gun74 l3gun72'}).text.strip()
        print("\n" + job_info[0])
        
        # Company Name
        # sometimes data-automation attribute is not working
#         company.append(job_post.find('a', attrs={'data-automation': 'jobCardCompanyLink'}).text.strip()) 
        job_info[1] = job_post.find('span', attrs={'class': 'z1s6m00 _17dyj7u1 _1hbhsw64u _1hbhsw60 _1hbhsw6r'}).text.strip()
        
        # Location and Salary
#         location = job_post.find('a', attrs={'data-automation': 'jobCardLocationLink'}).text
        loc_salary_info = [
            location_salary.getText(strip=True) for location_salary in job_post.findAll('span', attrs={'class': 'z1s6m00 _1hbhsw64u y44q7i0 y44q7i3 y44q7i21 y44q7ih'})
        ]
        job_info[2] = loc_salary_info[0]
        print(job_info[2])
        
        if len(loc_salary_info) == 2:
            job_info[3] = loc_salary_info[1]
            print(job_info[3])
        else:
            print("none")
        
        # Days since posted
        job_info[4] = job_post.find('time', attrs={'class': 'z1s6m00 _1hbhsw64u'}).text.strip()
        
        # Additional details
        details = job_post.find('a', attrs={'class': 'jdlu994 jdlu996 jdlu999 y44q7i2 z1s6m00 z1s6m0f _1hbhsw6h'}, href=True)

        # Job link
        job_post_url = 'https://www.jobstreet.com.ph' + str(details['href'])
        job_post_page = requests.get(job_post_url, headers=headers)
        job_info[5] = job_post_url
        
        job_post_soup = BeautifulSoup(job_post_page.content,'html.parser')
        job_post_soup_prettify = BeautifulSoup(job_post_soup.prettify(), 'html.parser')

        # Job skills and other description
        job_skills_desc(job_post_soup_prettify)
        
        # Job additional information
        job_addtl_info(job_post_soup_prettify)
                                                        
        # Save all additional information
        extracted_data.append(job_info)
        
# Saving the extracted data in excel
def jobs_save():
    # Save all information
    all_jobs_list = {} 
    
    # Transpose the data using numpy
    transposed_data = np.array(extracted_data).T.tolist()

    # Save the data using pandas
    for job in range(len(cols)):
        all_jobs_list[cols[job]] = transposed_data[job]

    filename = "jobstreet_dataset.xls"
    jobs = pd.DataFrame(all_jobs_list)
    jobs.to_excel(filename, index=False)    

# Job searching 
def jobs_search():
    page_number = 1 # Current page number of webscraper; Initial is 1
    total_num_pages = 1 # Number of pages to be extracted; Initial is 1
    jobs_per_page = 30 # Number of jobs per page

    while page_number <= total_num_pages:
        url = 'https://www.jobstreet.com.ph/en/job-search/data-analyst-jobs/' + str(page_number)
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content,"html.parser")
        soup_prettify = BeautifulSoup(soup.prettify(), "html.parser")

        job_list = soup_prettify.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw69e _1hbhsw68e _1hbhsw672 _1hbhsw67e'})
#         job_list = soup_prettify.findAll('div', attrs={'data-automation': 'jobListing'})

        # Get the number of jobs
        if page_number == 1:
            job_search_bar = soup_prettify.find('div', attrs={'data-automation': 'searchResultBar'}).text.split('of')
            job_numbers = int(job_search_bar[1].replace('jobs', '').replace(',', '').strip())
            total_num_pages = math.ceil(job_numbers/jobs_per_page)
            print(total_num_pages)
        
        print("Page number: " + str(page_number))
        page_number += 1
            
        jobs_extract(job_list)
    
    jobs_save()
                
jobs_search()

In [ ]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();